# Scratchwork for Dataset Sampling Method Engineering

In [2]:
import pandas as pd
import os
import requests
import sqlalchemy
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
import json

In [3]:
CWD = os.getcwd()
with open(f"{CWD}/example_response.json", encoding="utf8") as f:
    example_response = json.load(f)

In [4]:
example_response_df = pd.DataFrame(example_response["items"])
example_response_df.tail(2)

,kind,title,htmlTitle,link,displayLink,snippet,htmlSnippet,formattedUrl,htmlFormattedUrl,pagemap,cacheId
8,customsearch#result,Module 8 Michelangelo and Giacometti Forum | A...,Module 8 Michelangelo and Giacometti Forum | A...,https://courses.lumenlearning.com/atd-herkimer...,courses.lumenlearning.com,"Jul 8, 2008 ... ... [CC BY-SA 3.0 (https://cre...","Jul 8, 2008 <b>...</b> ... [CC BY-SA <b>3.0</b...",https://courses.lumenlearning.com/.../module-8...,<b>https</b>://courses.lumenlearning.com/.../m...,{'cse_thumbnail': [{'src': 'https://encrypted-...,pPsOnPunpu4J
9,customsearch#result,Aorta — Taming the SRU,Aorta — Taming the SRU,https://www.tamingthesru.com/aorta-module,www.tamingthesru.com,By Edoarado [CC BY-SA 3.0 (http://creativecomm...,By Edoarado [CC BY-SA <b>3.0</b> (<b>http</b>:...,https://www.tamingthesru.com/aorta-module,<b>https</b>://www.tamingthesru.com/aorta-module,{'cse_thumbnail': [{'src': 'https://encrypted-...,3RS0M2eZ578J


In [5]:
web_contents = requests.get(example_response_df.loc[0, "link"]).text
encoding = EncodingDetector.find_declared_encoding(web_contents, is_html = True)
soup = BeautifulSoup(web_contents, "lxml", from_encoding=encoding)
for script in soup(["script", "style"]):
    script.extract()
parse_result = soup.get_text("", strip = True)
print(f"title: {example_response_df.loc[0, 'title']}")
print(f"url: {example_response_df.loc[0, 'link']}")
print(f"contents\n: {parse_result[:500]}")

C:\Users\brand\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\bs4\__init__.py:226: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


title: Projects API | GitLab
url: https://docs.gitlab.com/ee/api/projects.html
contents
: Projects API | GitLabDocsWhat's new?Get free trialGitLab DocsLearn GitLab with tutorialsChoose a subscriptionGitLab SaaS subscriptionsSelf-managed subscriptionsGitLab Dedicated subscriptionsCompare self-managed to SaaSQuarterly reconciliationStorage usage quotaCI/CD minutes quotaFree user limitActivate Enterprise EditionFeatures available to Starter and Bronze subscribersInstall GitLabRequirementsPostgreSQL extensionsInstallation methodsLinux packages (Omnibus)ArchitectureOmnibus packages and im


In [6]:
for i in example_response_df.index:
    web_contents = requests.get(example_response_df.loc[i, "link"]).text
    encoding = EncodingDetector.find_declared_encoding(web_contents, is_html = True)
    soup = BeautifulSoup(web_contents, "lxml", from_encoding=encoding)
    for script in soup(["script", "style"]):
        script.extract()
    parse_result = soup.get_text("", strip = True)
    print(f"entry {i}")
    print(f"title: {soup.title}")
    '''print(f"url: {example_response_df.loc[i, 'link']}")
    print(f"contents\n: {parse_result[:500]}")'''

entry 0
title: <title>Projects API | GitLab</title>
entry 1
title: <title>How and why use Creative Commons licensed work | Opensource.com</title>
entry 2
title: <title>Least Open - Creative Commons Licenses Explained - LibGuides at University of Hartford</title>
entry 3
title: <title>Spiral: Efficient Reinforcement Learning using Gaussian Processes</title>
entry 4
title: <title>About / Abstrakt Reflections</title>
entry 5
title: <title>The Effects of Teacher-Student Relationships on the Academic Engagement of Students</title>
entry 6
title: <title>Database: TcoF-DB v2: update of the database of human and mouse transcription co-factors and transcription factor interactions</title>
entry 7
title: <title>Licence - FreeCAD Documentation</title>
entry 8
title: <title>Module 8  Michelangelo and Giacometti  Forum | Art Appreciation</title>
entry 9
title: <title>Aorta — Taming the SRU</title>


In [7]:
import sqlalchemy

In [8]:
engine = sqlalchemy.create_engine(f"sqlite:///{CWD}/example_sample.db")
engine.connect()

In [9]:
sample_df = example_response_df.loc[:, ["link"]]
sample_titles = []
sample_contents = []
for address in sample_df["link"]:
    web_contents = requests.get(address).text
    encoding = EncodingDetector.find_declared_encoding(web_contents, is_html = True)
    soup = BeautifulSoup(web_contents, "lxml", from_encoding=encoding)
    for script in soup(["script", "style"]):
        script.extract()
    parse_result = soup.get_text("", strip = True)
    sample_titles.append(soup.title)
    sample_contents.append(parse_result[:3000])
sample_df["title"] = sample_titles
sample_df["content"] = sample_contents
sample_df.head()

C:\Users\brand\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\bs4\__init__.py:226: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


,link,title,content
0,https://docs.gitlab.com/ee/api/projects.html,[Projects API | GitLab],Projects API | GitLabDocsWhat's new?Get free t...
1,https://opensource.com/article/20/1/what-creat...,[How and why use Creative Commons licensed wor...,How and why use Creative Commons licensed work...
2,https://libguides.hartford.edu/c.php?g=887688&...,[Least Open - Creative Commons Licenses Explai...,Least Open - Creative Commons Licenses Explain...
3,https://spiral.imperial.ac.uk/handle/10044/1/1...,[Spiral: Efficient Reinforcement Learning usin...,Spiral: Efficient Reinforcement Learning using...
4,https://www.abstraktreflections.net/about/,[About / Abstrakt Reflections],About / Abstrakt ReflectionsNEWSABOUTARTISTSRE...


In [10]:
sample_df["title"] = sample_df["title"].astype(str)

In [11]:
sample_df.to_sql("example", engine, if_exists = 'append')

10

In [12]:
pd.read_sql("SELECT * FROM example", engine)

,index,link,title,content
0,0,https://docs.gitlab.com/ee/api/projects.html,<title>Projects API | GitLab</title>,Projects API | GitLabDocsWhat's new?Get free t...
1,1,https://opensource.com/article/20/1/what-creat...,<title>How and why use Creative Commons licens...,How and why use Creative Commons licensed work...
2,2,https://libguides.hartford.edu/c.php?g=887688&...,<title>Least Open - Creative Commons Licenses ...,Least Open - Creative Commons Licenses Explain...
3,3,https://spiral.imperial.ac.uk/handle/10044/1/1...,<title>Spiral: Efficient Reinforcement Learnin...,Spiral: Efficient Reinforcement Learning using...
4,4,https://www.abstraktreflections.net/about/,<title>About / Abstrakt Reflections</title>,About / Abstrakt ReflectionsNEWSABOUTARTISTSRE...
5,5,https://mdsoar.org/handle/11603/3893,<title>The Effects of Teacher-Student Relation...,The Effects of Teacher-Student Relationships o...
6,6,https://repository.kaust.edu.sa/handle/10754/6...,<title>Database: TcoF-DB v2: update of the dat...,Database: TcoF-DB v2: update of the database o...
7,7,https://wiki.freecadweb.org/Licence,<title>Licence - FreeCAD Documentation</title>,Licence - FreeCAD DocumentationLicenceFrom Fre...
8,8,https://courses.lumenlearning.com/atd-herkimer...,<title>Module 8 Michelangelo and Giacometti ...,Module 8 Michelangelo and Giacometti Forum |...
9,9,https://www.tamingthesru.com/aorta-module,<title>Aorta — Taming the SRU</title>,Aorta — Taming the SRUTopTaming the SRUBlogEme...


In [13]:
cc_license_data = pd.read_csv(f"{CWD}/legal-tool-paths.txt", header=None)
license_pattern = r"((?:[^/]+/){2}(?:[^/]+)).*"
license_pattern_map = {
    "by": "licenses/by/",
    "by-sa": "licenses/by-sa/",
    "by-nc": "licenses/by-nc/",
    "by-nc-sa": "licenses/by-nc-sa/",
    "by-nd": "licenses/by-nd/",
    "by-nc-nd": "licenses/by-nc-nd/|licenses/by-nd-nc/",
    "publicdomain": "publicdomain/"
}
license_list = pd.Series(
    cc_license_data[0]
    .str.extract(license_pattern, expand=False)
    .dropna()
    .unique()
)
license_series_map = {
    k: license_list[license_list.str.contains(license_pattern_map[k])]
    for k in license_pattern_map
}
license_series_map

{'by': 0     licenses/by/1.0
 14    licenses/by/2.0
 27    licenses/by/2.1
 33    licenses/by/2.5
 39    licenses/by/3.0
 45    licenses/by/4.0
 dtype: object,
 'by-sa': 5     licenses/by-sa/1.0
 20    licenses/by-sa/2.0
 32    licenses/by-sa/2.1
 38    licenses/by-sa/2.5
 44    licenses/by-sa/3.0
 50    licenses/by-sa/4.0
 dtype: object,
 'by-nc': 1     licenses/by-nc/1.0
 15    licenses/by-nc/2.0
 28    licenses/by-nc/2.1
 34    licenses/by-nc/2.5
 40    licenses/by-nc/3.0
 46    licenses/by-nc/4.0
 dtype: object,
 'by-nc-sa': 2     licenses/by-nc-sa/1.0
 17    licenses/by-nc-sa/2.0
 30    licenses/by-nc-sa/2.1
 36    licenses/by-nc-sa/2.5
 42    licenses/by-nc-sa/3.0
 48    licenses/by-nc-sa/4.0
 dtype: object,
 'by-nd': 3     licenses/by-nd/1.0
 18    licenses/by-nd/2.0
 31    licenses/by-nd/2.1
 37    licenses/by-nd/2.5
 43    licenses/by-nd/3.0
 49    licenses/by-nd/4.0
 dtype: object,
 'by-nc-nd': 4     licenses/by-nd-nc/1.0
 16    licenses/by-nc-nd/2.0
 19    licenses/by-nd-nc/

In [32]:
import dataset_sampling
license_map = dataset_sampling.get_license_map()
license_ser = pd.concat([v for v in license_map.values()])
license_ser_splits_df = license_ser.str.split("/", expand=True)
license_ser_splits_df = license_ser_splits_df.rename(
    columns = {
        0: "Tool Typing",
        1: "General Typing",
        2: "Version",
        3: "Jurisdiction"
    }
)
license_ser_splits_df["General Typing"] = license_ser_splits_df["General Typing"].str.replace("mark|zero", "publicdomain", regex=True)
license_ser_splits_df["General Typing"] = license_ser_splits_df["General Typing"].str.replace("by-nd-nc", "by-nc-nd", regex=True)
license_ser_splits_df["Version"] = license_ser_splits_df["Version"].astype(float)

In [33]:
license_one_hot_encoding = pd.DataFrame()
license_one_hot_encoding["by"] = license_ser_splits_df["General Typing"].str.contains("by")
license_one_hot_encoding["sa"] = license_ser_splits_df["General Typing"].str.contains("sa")
license_one_hot_encoding["nc"] = license_ser_splits_df["General Typing"].str.contains("nc")
license_one_hot_encoding["nd"] = license_ser_splits_df["General Typing"].str.contains("nd")
license_not_six_type = license_ser_splits_df["General Typing"].str.contains("by|sa|nc|nd")
license_one_hot_encoding["neither"] = ~(license_not_six_type.fillna(False))

In [34]:
license_df = pd.concat([license_ser, license_ser_splits_df, license_one_hot_encoding], axis = 1)\
    .rename(columns = {0: "license"})

In [35]:
sampling_engine = sqlalchemy.create_engine(f"sqlite:///{CWD}/modeling_dataset.db")
sampling_engine.connect()

In [36]:
pd.read_sql("SELECT * FROM sqlite_master WHERE type='table'", sampling_engine)

,type,name,tbl_name,rootpage,sql
0,table,by,by,2,"CREATE TABLE ""by"" (\n\t""index"" BIGINT, \n\tlic..."
1,table,by-sa,by-sa,404,"CREATE TABLE ""by-sa"" (\n\t""index"" BIGINT, \n\t..."
2,table,by-nc,by-nc,820,"CREATE TABLE ""by-nc"" (\n\t""index"" BIGINT, \n\t..."
3,table,by-nc-sa,by-nc-sa,1220,"CREATE TABLE ""by-nc-sa"" (\n\t""index"" BIGINT, \..."
4,table,by-nd,by-nd,1223,"CREATE TABLE ""by-nd"" (\n\t""index"" BIGINT, \n\t..."
5,table,by-nc-nd,by-nc-nd,1590,"CREATE TABLE ""by-nc-nd"" (\n\t""index"" BIGINT, \..."
6,table,publicdomain,publicdomain,2010,"CREATE TABLE publicdomain (\n\t""index"" BIGINT,..."


In [37]:
tables = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table'", sampling_engine)
inspected_dataset = pd.concat(
    [
        pd.read_sql(
            f"SELECT * FROM '{license_type}'",
            sampling_engine
        )
        for license_type in tables["name"]
    ]
)
inspected_dataset.groupby("url").agg(lambda x: x.iloc[0])

,index,license,title,contents
url,,,,
http://archivescentral.org.nz/rights/CC-BY-NC-ND-4.0,42,licenses/by-nd/4.0,<title>Creative Commons 4.0: Attribution - Non...,Creative Commons 4.0: Attribution - Non-Commer...
http://ccmixter.org/how-to-attribute-ccmixter-tracks,0,licenses/by/3.0,<title>ccMixter - How to Attribute ccMixter Tr...,ccMixter - How to Attribute ccMixter Tracksski...
http://cor.esipfed.org/pylode?url=http://cor.esipfed.org/ont/earthcube/cclo,30,licenses/by-nc-sa/2.0,<title>Creative Commons Licenses Ontology</title>,Creative Commons Licenses Ontologymade bypyLOD...
http://docs.fedoraproject.org/en-US/Fedora/12/html/Accessibility_Guide/index.html,48,licenses/by-sa/2.1,<title>Accessibility Guide</title>,"Accessibility GuideThis is an iframe, to view ..."
http://dro.dur.ac.uk/27362,68,licenses/by/3.0,<title>Mantle plumes and mantle dynamics in th...,Mantle plumes and mantle dynamics in the Wilso...
...,...,...,...,...
https://yorkspace.library.yorku.ca/xmlui/handle/10315/35637?show=full,79,licenses/by/2.5,<title>Exclosure of patch of grass at Camp Fin...,"Exclosure of patch of grass at Camp Finney, Wa..."
https://yorkspace.library.yorku.ca/xmlui/handle/10315/37157?show=full,33,licenses/by-nd/2.5,<title>Art and Pragmatism: James and Dewey on ...,Art and Pragmatism: James and Dewey on the Rec...
https://yorkspace.library.yorku.ca/xmlui/handle/10315/37602?show=full,58,licenses/by-nc/2.5,"<title>Day 3 Panel 1 Gender and Identity, Pane...","Day 3 Panel 1 Gender and Identity, Panel Chair..."


In [49]:
inspected_dataset_license = inspected_dataset.merge(license_df, on = "license")
inspected_dataset_license["content_sum"] = \
    inspected_dataset_license["url"] + \
    inspected_dataset_license["title"] + \
    inspected_dataset_license["contents"]
inspected_dataset_license = inspected_dataset_license.groupby("content_sum").agg(lambda x: x.iloc[0])
inspected_dataset_license.groupby("General Typing").count()

,index,license,url,title,contents,Tool Typing,Version,by,sa,nc,nd,neither
General Typing,,,,,,,,,,,,
by,419,419,419,419,419,419,419,419,419,419,419,419
by-nc,249,249,249,249,249,249,249,249,249,249,249,249
by-nc-nd,123,123,123,123,123,123,123,123,123,123,123,123
by-nc-sa,213,213,213,213,213,213,213,213,213,213,213,213
by-nd,137,137,137,137,137,137,137,137,137,137,137,137
by-sa,250,250,250,250,250,250,250,250,250,250,250,250
publicdomain,160,160,160,160,160,160,160,160,160,160,160,160


In [50]:
inspected_dataset_license[inspected_dataset_license["General Typing"] == "by-nc-nd"]

,index,license,url,title,contents,Tool Typing,General Typing,Version,by,sa,nc,nd,neither
content_sum,,,,,,,,,,,,,
"http://ccmixter.org/how-to-attribute-ccmixter-tracks<title>ccMixter - How to Attribute ccMixter Tracks</title>ccMixter - How to Attribute ccMixter TracksskipNot logged inLog InSearchFind contentDownload, Sample, Cut-up, Share.HomePicksRemixesSamplesA CappellasPeopleExtrasPlaylistsHow to Attribute ccMixter TracksHow to Attribute ccMixer TracksCreative Commons Tracks are Free - When You Give CreditccMixter.orgoffers over 30,000 original tracks that can be used freely in YouTube videos, remixes, mobile apps, games, and more. Depending on which Creative Commons license the artist has designated, some tracks can even beused commericallyÂ (CC BYÂ but notCC BY NC)! The only requirement?Attribution.Â ÂÂ Â Â Â Â Â Â Â Â Â ÂAllows Commercial Use Â Â Â Â Â Noncomercial Use OnlyHere's a tutorial video shows how simple it is to Attribute. Check it out!How do you Attribute?1. GIVE CREDIT.Â Be sure to give credit in the video or app or game itself. Attribution is currency! For example, if you usecafe connection by morgantj,Â Attribute in the actual video or game or app:ÂMusic ""cafe connection"" by morgantj featuring MorusqueAvailable at ccMixter.org http://ccmixter.org/files/morgantj/18947Under CC BY license http://creativecommons.org/licenses/by/3.0/2. LINK BACK.Attribution requires you link back and helps the artist get credit. For example, in the YouTube description field or on the webpage where you embed your video or have folks download your app, the Attribution link should like this:Music ""cafe connection"" by morgantj featuring MorusqueAvailable at ccMixter.org http://ccmixter.org/files/morgantj/18947Under CC BY license http://creativecommons.org/licenses/by/3.0/3. NAME THE LICENSE.Include the type of the CC license designed for the track, and link to it. The examples above do that on the last line. This explains to others how CC licenses work.ÂIt is simple to utilize the free music library atccMixter.org, sostart digging! Just remember, Attribution -- giving credit -- is what makes it possible for these artists to give their music away. Yep, Attribution is the new currency.ÂVisitorsFind MusicForumsAboutLooking for...?TwitterArtistsLog InRegisterSearch our archives formusic for your video,podcast or school projectatdig.ccMixterSupport ccMixterEditors' PicksA Motherless ...Hanging Eleve...Blow Your Hor...The creatures...Mary Was A Sp...More picks...Highest RatedGeneric Ap...Blow Your ...The Inner ...Chill This...Apologies ...On the Bri...Recent ReviewersAdmiral BobBOCrewRizKeyGApoxodeKara SquareSpeckZenboy1955More reviews...This text and images on this site are licensed under aCreative Commons Attribution-Noncommercial 3.0 United States License.This site usesccHost, licensed underCC-GNU-GPL, which is a product of theccToolsproject and usesGetID3. This site is sponsored byArtisTech Media. By using this site, you agree to ourTerms of UseandPrivacy Policy.Please note oursite credits. Contact:site administrator.",28,licenses/by-nc-nd/3.0,http://ccmixter.org/how-to-attribute-ccmixter-...,<title>ccMixter - How to Attribute ccMixter Tr...,ccMixter - How to Attribute ccMixter Tracksski...,licenses,by-nc-nd,3.0,True,False,True,True,False
"http://eprints.biblio.unitn.it/1964/<title>The Regulatory Anticommons of Green Infrastructures Unitn Eprints Research</title>The Regulatory Anticommons of Green Infrastructures Unitn Eprints ResearchUniversità degli Studi di TrentoUnitn-eprints.PhDHomeAboutUser areaHelpAbout Trento UniversityUniversity LibraryUnitn-eprints PhDLoginThe Regulatory Anticommons of Green InfrastructuresBellantuono, Giuseppe(2011)The Regulatory Anticommons of Green Infrastructures.Trento Law and Technology Research Group Research Papers; 5; 5 . Trento: Università degli Studi di Trento. ISBN 978-88-8443-362-6PreviewPDFDownload (459Kb)|PreviewAbstractDevelopment of green infrastructures (renewable energy plants and tran

In [42]:
inspected_dataset_license["parsed_title"] = \
    inspected_dataset_license["title"].str.extract(r"<title>(.*)</title>")
inspected_dataset_license

,index,license,title,contents,Tool Typing,General Typing,Version,by,sa,nc,nd,neither,parsed_title
url,,,,,,,,,,,,,
http://archivescentral.org.nz/rights/CC-BY-NC-ND-4.0,42,licenses/by-nd/4.0,<title>Creative Commons 4.0: Attribution - Non...,Creative Commons 4.0: Attribution - Non-Commer...,licenses,by-nd,4.0,True,False,False,True,False,Creative Commons 4.0: Attribution - Non-Commer...
http://ccmixter.org/how-to-attribute-ccmixter-tracks,0,licenses/by/3.0,<title>ccMixter - How to Attribute ccMixter Tr...,ccMixter - How to Attribute ccMixter Tracksski...,licenses,by,3.0,True,False,False,False,False,ccMixter - How to Attribute ccMixter Tracks
http://cor.esipfed.org/pylode?url=http://cor.esipfed.org/ont/earthcube/cclo,30,licenses/by-nc-sa/2.0,<title>Creative Commons Licenses Ontology</title>,Creative Commons Licenses Ontologymade bypyLOD...,licenses,by-nc-sa,2.0,True,True,True,False,False,Creative Commons Licenses Ontology
http://docs.fedoraproject.org/en-US/Fedora/12/html/Accessibility_Guide/index.html,48,licenses/by-sa/2.1,<title>Accessibility Guide</title>,"Accessibility GuideThis is an iframe, to view ...",licenses,by-sa,2.1,True,True,False,False,False,Accessibility Guide
http://dro.dur.ac.uk/27362,68,licenses/by/3.0,<title>Mantle plumes and mantle dynamics in th...,Mantle plumes and mantle dynamics in the Wilso...,licenses,by,3.0,True,False,False,False,False,Mantle plumes and mantle dynamics in the Wilso...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://yorkspace.library.yorku.ca/xmlui/handle/10315/35637?show=full,79,licenses/by/2.5,<title>Exclosure of patch of grass at Camp Fin...,"Exclosure of patch of grass at Camp Finney, Wa...",licenses,by,2.5,True,False,False,False,False,"Exclosure of patch of grass at Camp Finney, Wa..."
https://yorkspace.library.yorku.ca/xmlui/handle/10315/37157?show=full,33,licenses/by-nd/2.5,<title>Art and Pragmatism: James and Dewey on ...,Art and Pragmatism: James and Dewey on the Rec...,licenses,by-nd,2.5,True,False,False,True,False,Art and Pragmatism: James and Dewey on the Rec...
https://yorkspace.library.yorku.ca/xmlui/handle/10315/37602?show=full,58,licenses/by-nc/2.5,"<title>Day 3 Panel 1 Gender and Identity, Pane...","Day 3 Panel 1 Gender and Identity, Panel Chair...",licenses,by-nc,2.5,True,False,True,False,False,"Day 3 Panel 1 Gender and Identity, Panel Chair..."
